In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, matthews_corrcoef, roc_auc_score, confusion_matrix, f1_score

import optuna

import psycopg2
from config import *

from datetime import timedelta

import utm
import time

from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import class_weight
import optuna

In [2]:
conexion = psycopg2.connect(database=db_database, 
                                user=db_user, 
                                password=db_password, 
                                host=db_host, 
                                port=db_port)

In [3]:
fechaHasta='2023-01-01'
pathModelos = '/data/proyectos/GRAPEVINE/Models/fjlacueva/Phenology/RandomForests'
query="select * from public.sabana_sin_rad where fecha <= '"+ fechaHasta + "';"
sabana = pd.read_sql_query(query, con=conexion).drop_duplicates()
sabana

,phenologystageid,variedad,codigo,longitude,latitude,altitud,doc_id,fecha,estacion,season,...,do_Borja,do_Calatayud,do_Cariñena,do_Somontano,variedad_CABERNET SAUVIGNON,variedad_CHARDONNAY,variedad_GARNACHA,variedad_MAZUELA,variedad_SYRACH,variedad_TEMPRANILLO
0,10.0,MAZUELA,50073A08600010,-1.179190,41.354589,592.0,Cariñena,2022-09-26,gv01,2021_2022,...,0,0,1,0,0,0,0,1,0,0
1,10.0,CABERNET SAUVIGNON,50144A02900008,-1.195923,41.401890,506.0,Cariñena,2022-09-27,gv01,2021_2022,...,0,0,1,0,1,0,0,0,0,0
2,10.0,GARNACHA,50144A02300023,-1.176083,41.371589,580.0,Cariñena,2022-09-27,gv01,2021_2022,...,0,0,1,0,0,0,1,0,0,0
3,10.0,SYRACH,50073A08400031,-1.207635,41.355942,565.0,Cariñena,2022-09-27,gv01,2021_2022,...,0,0,1,0,0,0,0,0,1,0
4,10.0,GARNACHA,50073A00500022,-1.208139,41.397524,507.0,Cariñena,2022-09-27,gv01,2021_2022,...,0,0,1,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104630,10.0,MAZUELA,50073A08600010,-1.179190,41.354589,592.0,Cariñena,2022-09-25,gv01,2021_2022,...,0,0,1,0,0,0,0,1,0,0
104631,10.0,CABERNET SAUVIGNON,50144A02900008,-1.195923,41.401890,506.0,Cariñena,2022-09-26,gv01,2021_2022,...,0,0,1,0,1,0,0,0,0,0
104632,10.0,GARNACHA,50144A02300023,-1.176083,41.371589,580.0,Cariñena,2022-09-26,gv01,2021_2022,...,0,0,1,0,0,0,1,0,0,0
104633,10.0,SYRACH,50073A08400031,-1.207635,41.355942,565.0,Cariñena,2022-09-26,gv01,2021_2022,...,0,0,1,0,0,0,0,0,1,0


In [4]:
sabana['doc_id']=sabana.doc_id.fillna('New')

In [5]:
sabana.doc_id.value_counts()

Cariñena     68387
Borja        28918
Calatayud     7141
Somontano      189
Name: doc_id, dtype: int64

In [6]:
parcelas_train=[]
parcelas_test=[]
parcelas_vali=[]
for do in sabana.doc_id.unique():
    print(do)
    parcelas_do_train, parcelas_do_no=train_test_split(list(sabana[sabana.doc_id==do].codigo.unique()),
                                                       random_state=17,test_size=0.4)
    parcelas_train=parcelas_train+parcelas_do_train
    if len(parcelas_do_no)>1:
        parcelas_do_vali, parcelas_do_test=train_test_split(parcelas_do_no,test_size=0.5, random_state=17)
        parcelas_train=parcelas_train+parcelas_do_train
        parcelas_test=parcelas_test+parcelas_do_test
        parcelas_vali=parcelas_vali+parcelas_do_vali
    else:
        pascelas_test=parcelas_test+parcelas_do_no
        

Cariñena
Borja
Calatayud
Somontano


In [7]:
parcelas_vali

['50201A02100075',
 '50004A03400175',
 '50018A03200061',
 '50182A01500002',
 '50018A00400191',
 '50168A00900067',
 '50073A08900011',
 '50144A50100103',
 '50024A01200100',
 '50098A00400017',
 '50004A03200057',
 '50114A02000145',
 '50154A50100079',
 '50114A01900153',
 '50006A04000055',
 '50055A03500004',
 '50055A06200053',
 '50046A01700135',
 '50170A01000234',
 '50170A01800046']

In [8]:
conexion.close()

In [9]:
sabana.shift(-7)

,phenologystageid,variedad,codigo,longitude,latitude,altitud,doc_id,fecha,estacion,season,...,do_Borja,do_Calatayud,do_Cariñena,do_Somontano,variedad_CABERNET SAUVIGNON,variedad_CHARDONNAY,variedad_GARNACHA,variedad_MAZUELA,variedad_SYRACH,variedad_TEMPRANILLO
0,10.0,GARNACHA,50144A02300023,-1.176083,41.371589,580.0,Cariñena,2022-09-28,gv01,2021_2022,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,10.0,SYRACH,50073A08400031,-1.207635,41.355942,565.0,Cariñena,2022-09-28,gv01,2021_2022,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,10.0,GARNACHA,50073A00500022,-1.208139,41.397524,507.0,Cariñena,2022-09-28,gv01,2021_2022,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,10.0,MAZUELA,50073A08600010,-1.179190,41.354589,592.0,Cariñena,2022-09-28,gv01,2021_2022,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,10.0,CABERNET SAUVIGNON,50144A02900008,-1.195923,41.401890,506.0,Cariñena,2022-09-29,gv01,2021_2022,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104630,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
104631,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
104632,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
104633,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
temp=sabana.copy()
datos_list=[]
for id_terr in temp.codigo.unique():
    datos_camp_terr=temp[temp['codigo']==id_terr]
    datos_camp_terr['next_fecha']=datos_camp_terr['fecha'].shift(-7)
    datos_camp_terr['next_pheno']=datos_camp_terr['phenologystageid'].shift(-7)
    datos_camp_terr=datos_camp_terr[(datos_camp_terr.next_fecha-datos_camp_terr.fecha).dt.days==7]
    datos_list.append(datos_camp_terr)
datos_total=pd.concat(datos_list).reset_index(drop=True)
datos_total

/home/glabata/.conda/envs/grapevine/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/home/glabata/.conda/envs/grapevine/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/glabata/.conda/envs/grapevine/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


/home/glabata/.conda/envs/grapevine/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/home/glabata/.conda/envs/grapevine/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/glabata/.conda/envs/grapevine/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


/home/glabata/.conda/envs/grapevine/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/glabata/.conda/envs/grapevine/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/home/glabata/.conda/envs/grapevine/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


/home/glabata/.conda/envs/grapevine/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/home/glabata/.conda/envs/grapevine/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/glabata/.conda/envs/grapevine/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


/home/glabata/.conda/envs/grapevine/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/home/glabata/.conda/envs/grapevine/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/glabata/.conda/envs/grapevine/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


/home/glabata/.conda/envs/grapevine/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/home/glabata/.conda/envs/grapevine/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/glabata/.conda/envs/grapevine/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


/home/glabata/.conda/envs/grapevine/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/home/glabata/.conda/envs/grapevine/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/glabata/.conda/envs/grapevine/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


,phenologystageid,variedad,codigo,longitude,latitude,altitud,doc_id,fecha,estacion,season,...,do_Cariñena,do_Somontano,variedad_CABERNET SAUVIGNON,variedad_CHARDONNAY,variedad_GARNACHA,variedad_MAZUELA,variedad_SYRACH,variedad_TEMPRANILLO,next_fecha,next_pheno
0,3.0,MAZUELA,50073A08600010,-1.179190,41.354589,592.0,Cariñena,2022-05-20,gv01,2021_2022,...,1,0,0,0,0,1,0,0,2022-05-27,3.0
1,3.0,MAZUELA,50073A08600010,-1.179190,41.354589,592.0,Cariñena,2022-05-21,gv01,2021_2022,...,1,0,0,0,0,1,0,0,2022-05-28,3.0
2,3.0,MAZUELA,50073A08600010,-1.179190,41.354589,592.0,Cariñena,2022-05-22,gv01,2021_2022,...,1,0,0,0,0,1,0,0,2022-05-29,3.0
3,3.0,MAZUELA,50073A08600010,-1.179190,41.354589,592.0,Cariñena,2022-05-23,gv01,2021_2022,...,1,0,0,0,0,1,0,0,2022-05-30,3.0
4,3.0,MAZUELA,50073A08600010,-1.179190,41.354589,592.0,Cariñena,2022-05-24,gv01,2021_2022,...,1,0,0,0,0,1,0,0,2022-05-31,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80464,10.0,GARNACHA,44033A00100095,-1.361989,41.047367,824.0,Calatayud,2022-11-07,Z18,2021_2022,...,0,0,0,0,1,0,0,0,2022-11-14,10.0
80465,10.0,GARNACHA,44033A00100095,-1.361989,41.047367,824.0,Calatayud,2022-11-08,Z18,2021_2022,...,0,0,0,0,1,0,0,0,2022-11-15,10.0
80466,10.0,GARNACHA,44033A00100095,-1.361989,41.047367,824.0,Calatayud,2022-11-09,Z18,2021_2022,...,0,0,0,0,1,0,0,0,2022-11-16,10.0
80467,10.0,GARNACHA,44033A00100095,-1.361989,41.047367,824.0,Calatayud,2022-11-10,Z18,2021_2022,...,0,0,0,0,1,0,0,0,2022-11-17,10.0


In [11]:
def objective(trial,train,vali):
    variedades=[col for col in train.columns if 'variedad_' in col]
    dos=[col for col in train.columns if 'do_' in col]
    variables_basic=['phenologystageid','dia', 'min', 'max', 'mean', 'std', 'median','diff']+variedades+dos
    medidas=variables_basic
    
    altitud=trial.suggest_categorical('Altitud',[True,False])
    latitud=trial.suggest_categorical('Latitud',[True,False])
    longitud=trial.suggest_categorical('Longitud',[True,False])
    if(longitud):
        medidas.append('longitude')
    if(latitud):
        medidas.append('latitude')
    if(altitud):
        medidas.append('altitud')
        
    if (trial.suggest_categorical('Acumuladas',[True,False])):
        
        inicio=trial.suggest_categorical('Inicio mediciones',['t0','1','2'])
        suffix=''
        name_vars=[]
        
        chilling=trial.suggest_categorical('Chilling',[True,False])
        winkler=trial.suggest_categorical('Winkler',[True,False])
        gdd=trial.suggest_categorical('gdd',[True,False])
        
        acumulativo=trial.suggest_categorical('Acumulativo',[True,False])
        if (acumulativo):
            suffix='_Cumm'
            if(trial.suggest_categorical('Precipitaciones',[True,False])):
                name_vars.append('precip__'+inicio+'_')
                
        if (winkler|gdd):
            tbase=trial.suggest_categorical('Tbase',['Tbase','TbaseMax'])
            temperatura_inicio=trial.suggest_categorical('Temperatura inicio',['10.0','4.5'])
        if (chilling):
            tbase_chill=trial.suggest_categorical('Tbase_chilling',['Tbase','Tbasemin','Utah'])
            name_vars.append('chillingDD_7.0_'+inicio+'_' + tbase_chill +'_sum')
        if (winkler):
            if acumulativo:
                name_vars.append('winkler_'+temperatura_inicio+'_'+inicio+'_' + tbase)
            else:
                name_vars.append('winkler_'+temperatura_inicio+'_' + tbase)
        if (gdd):
            name_vars.append('gdd_'+temperatura_inicio+'_'+inicio+'_' + tbase+'_sum')
       
        if len(name_vars)>0:
            semanas_list=[]
            for i in range(1,3):
                if(trial.suggest_categorical(str(i)+'_semanas_atras',[True,False])):
                    semanas_list.append(' '+str(i)+'_semanas_atras')
            if(trial.suggest_categorical('1_semanas_adelante',[True,False])):
                    semanas_list.append(' 1_semanas_adelante')
                    
            for name_var in name_vars:
                medidas.append(name_var+suffix)
                for semana in semanas_list:
                    medidas.append(name_var+suffix+semana)
        
    else:
        name_vars=[]    
        if(trial.suggest_categorical('Temperatura',[True,False])):
            if(trial.suggest_categorical('Temperatura_media',[True,False])):
                name_vars.append('tmed_mean')
            if(trial.suggest_categorical('Temperatura_min',[True,False])):
                name_vars.append('tmed_min')
            if(trial.suggest_categorical('Temperatura_max',[True,False])):
                name_vars.append('tmed_max')

        if(trial.suggest_categorical('Viento',[True,False])):
            if(trial.suggest_categorical('Viento_norte',[True,False])):
                name_vars.append('wind_N')
            if(trial.suggest_categorical('Viento_noreste',[True,False])):
                name_vars.append('wind_NE')
            if(trial.suggest_categorical('Viento_este',[True,False])):
                name_vars.append('wind_E')
            if(trial.suggest_categorical('Viento_sureste',[True,False])):
                name_vars.append('wind_SE')
            if(trial.suggest_categorical('Viento_sur',[True,False])):
                name_vars.append('wind_S')
            if(trial.suggest_categorical('Viento_suroeste',[True,False])):
                name_vars.append('wind_SW')
            if(trial.suggest_categorical('Viento_oeste',[True,False])):
                name_vars.append('wind_W')
            if(trial.suggest_categorical('Viento_noroeste',[True,False])):
                name_vars.append('wind_NW')
        
        if len(name_vars)>0:
            dias_vars=[]
            for i in range(1,14):
                if(trial.suggest_categorical(str(i)+'_dias_atras',[True,False])):
                    dias_vars.append(' ' + str(i) + '_dias_atras')
            for i in range(1,7):
                if(trial.suggest_categorical(str(i)+'_dias_adelante',[True,False])):
                    dias_vars.append(' ' + str(i) + '_dias_adelante')

            for name_var in name_vars:
                medidas.append(name_var)
                for dia_var in dias_vars:
                    medidas.append(name_var+dia_var)

    medidas.append('next_pheno')
    train=train[medidas].dropna()
    vali=vali[medidas].dropna()
    
    print(len(medidas))

    X_train=train.drop(['next_pheno'], axis=1).values
    Y_train=train['next_pheno'].values
    X_vali=vali.drop(['next_pheno'], axis=1).values
    Y_vali=vali['next_pheno'].values

    params = {'random_state':17,'n_jobs':-1,
              'criterion':trial.suggest_categorical(f'criterion',['gini','entropy']),
              'class_weight':'balanced',
              'max_depth':trial.suggest_int(f'max_depth',low=1,high=20),
              'min_samples_leaf':float(trial.suggest_discrete_uniform(f'min_samples_leaf', low=0.05,high=0.2,q=0.01)),
              'n_estimators':int(trial.suggest_discrete_uniform(f'n_estimators', low=10,high=200,q=10))}
    
    model=RandomForestClassifier(**params) 
    model.fit(X_train,Y_train)
    preds=model.predict(X_vali)
    
    mcc=matthews_corrcoef(Y_vali,preds)
    print("MCC: ",mcc)
    print("Accuracy: ", accuracy_score(Y_vali,preds))
    SCORE=mcc
    
    return mcc

In [ ]:
random_state  = 17
direction     = 'maximize'
n_trials      = 5
n_jobs        = 1
timeout       = None
verbosity     = 0
trozos=1000



train=datos_total[datos_total.codigo.isin(parcelas_train)]
vali=datos_total[datos_total.codigo.isin(parcelas_vali)]
train=train.drop(columns=['codigo','fecha'])
print(len(train))
vali=vali.drop(columns=['codigo','fecha'])

study_name = f'Fenologia'
sampler    = optuna.samplers.TPESampler(seed=random_state)

FILE = pathModelos+ f'/resumen_optuna-fenologia-rf-mcc-parcelas.csv'

study = optuna.create_study(study_name=study_name,direction=direction,sampler=sampler,
                            storage='sqlite:////'+pathModelos+'/optuna-aceituna-fenologia-rf-mcc-parcelas.db', 
                            load_if_exists=True)

for j in range(trozos):
    study.optimize(func=lambda trial: objective(trial,train,vali),n_trials=n_trials,timeout=timeout)

    result_df = study.trials_dataframe()
    result_df.to_csv(FILE)
    time.sleep(1)

[I 2023-05-02 18:32:06,096] Using an existing study with name 'Fenologia' instead of creating a new one.


43571
27


[I 2023-05-02 18:32:14,856] Trial 5000 finished with value: 0.8233794117920181 and parameters: {'Altitud': True, 'Latitud': True, 'Longitud': False, 'Acumuladas': True, 'Inicio mediciones': '1', 'Chilling': True, 'Winkler': True, 'gdd': False, 'Acumulativo': True, 'Precipitaciones': True, 'Tbase': 'TbaseMax', 'Temperatura inicio': '4.5', 'Tbase_chilling': 'Tbase', '1_semanas_atras': True, '2_semanas_atras': False, '1_semanas_adelante': False, 'criterion': 'gini', 'max_depth': 4, 'min_samples_leaf': 0.05, 'n_estimators': 30.0}. Best is trial 2322 with value: 0.8370929659105292.


MCC:  0.8233794117920181
Accuracy:  0.8649101925990046
27


[I 2023-05-02 18:32:16,638] Trial 5001 finished with value: 0.8255408999666954 and parameters: {'Altitud': True, 'Latitud': True, 'Longitud': False, 'Acumuladas': True, 'Inicio mediciones': '1', 'Chilling': True, 'Winkler': True, 'gdd': False, 'Acumulativo': True, 'Precipitaciones': True, 'Tbase': 'TbaseMax', 'Temperatura inicio': '4.5', 'Tbase_chilling': 'Tbase', '1_semanas_atras': True, '2_semanas_atras': False, '1_semanas_adelante': False, 'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 0.05, 'n_estimators': 70.0}. Best is trial 2322 with value: 0.8370929659105292.


...

MCC:  0.7754822757342291
Accuracy:  0.8290954338887687
27


[I 2023-05-02 18:32:28,641] Trial 5008 finished with value: 0.8256013988470078 and parameters: {'Altitud': True, 'Latitud': True, 'Longitud': False, 'Acumuladas': True, 'Inicio mediciones': '1', 'Chilling': True, 'Winkler': True, 'gdd': False, 'Acumulativo': True, 'Precipitaciones': True, 'Tbase': 'TbaseMax', 'Temperatura inicio': '4.5', 'Tbase_chilling': 'Utah', '1_semanas_atras': True, '2_semanas_atras': False, '1_semanas_adelante': False, 'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 0.05, 'n_estimators': 80.0}. Best is trial 2322 with value: 0.8370929659105292.


MCC:  0.8256013988470078
Accuracy:  0.86637091538628
27


[I 2023-05-02 18:32:29,952] Trial 5009 finished with value: 0.8254452257460473 and parameters: {'Altitud': True, 'Latitud': True, 'Longitud': False, 'Acumuladas': True, 'Inicio mediciones': '1', 'Chilling': True, 'Winkler': True, 'gdd': False, 'Acumulativo': True, 'Precipitaciones': True, 'Tbase': 'TbaseMax', 'Temperatura inicio': '4.5', 'Tbase_chilling': 'Tbase', '1_semanas_atras': True, '2_semanas_atras': False, '1_semanas_adelante': False, 'criterion': 'gini', 'max_depth': 6, 'min_samples_leaf': 0.05, 'n_estimators': 60.0}. Best is trial 2322 with value: 0.8370929659105292.


MCC:  0.8254452257460473
Accuracy:  0.86637091538628
